In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnan, count, isnull, avg, round, sum, desc, row_number
from pyspark.sql.window import Window
import pandas as pd
import io

print("VIDEO GAMES SALES ANALYSIS")

csv_200 = """Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
1,"Wii Sports",Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,"Super Mario Bros.",NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,"Mario Kart Wii",Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,"Wii Sports Resort",Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33
5,"Pokemon Red/Pokemon Blue",GB,1996,"Role-Playing",Nintendo,11.27,8.89,10.22,1,31.37
6,Tetris,GB,1989,Puzzle,Nintendo,23.2,2.26,4.22,0.58,30.26
7,"New Super Mario Bros.",DS,2006,Platform,Nintendo,11.38,9.23,6.5,2.9,30.01
8,"Wii Play",Wii,2006,Misc,Nintendo,14.03,9.2,2.93,2.85,29.02
9,"New Super Mario Bros. Wii",Wii,2009,Platform,Nintendo,14.59,7.06,4.7,2.26,28.62
10,"Duck Hunt",NES,1984,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31
11,Nintendogs,DS,2005,Simulation,Nintendo,9.07,11,1.93,2.75,24.76
12,"Mario Kart DS",DS,2005,Racing,Nintendo,9.81,7.57,4.13,1.92,23.42
13,"Pokemon Gold/Pokemon Silver",GB,1999,"Role-Playing",Nintendo,9,6.18,7.2,0.71,23.1
14,"Wii Fit",Wii,2007,Sports,Nintendo,8.94,8.03,3.6,2.15,22.72
15,"Wii Fit Plus",Wii,2009,Sports,Nintendo,9.09,8.59,2.53,1.79,22
16,"Kinect Adventures!",X360,2010,Misc,"Microsoft Game Studios",14.97,4.94,0.24,1.67,21.82
17,"Grand Theft Auto V",PS3,2013,Action,"Take-Two Interactive",7.01,9.27,0.97,4.14,21.4
18,"Grand Theft Auto: San Andreas",PS2,2004,Action,"Take-Two Interactive",9.43,0.4,0.41,1.02,11.26
19,"Super Mario World",SNES,1990,Platform,Nintendo,12.78,3.75,3.54,0.55,20.61
20,"Brain Age: Train Your Brain in Minutes a Day",DS,2005,Misc,Nintendo,4.75,9.26,4.16,2.05,20.22
21,"Pokemon Diamond/Pokemon Pearl",DS,2006,"Role-Playing",Nintendo,6.42,4.52,6.04,1.37,18.36
22,"Super Mario Land",GB,1989,Platform,Nintendo,10.83,2.71,4.14,0.42,18.14
23,"Super Mario Bros. 3",SNES,1988,Platform,Nintendo,9.54,3.44,3.84,0.46,17.28
24,"Gran Turismo 3: A-Spec",PS2,2001,Racing,"Sony Computer Entertainment",6.85,5.09,1.87,1.16,14.98
25,"Call of Duty: Modern Warfare 3",X360,2011,Shooter,Activision,9.03,4.28,0.13,1.32,14.76
26,"Grand Theft Auto V",X360,2013,Action,"Take-Two Interactive",9.63,5.31,0.06,1.38,16.38
27,"Animal Crossing: Wild World",DS,2005,Simulation,Nintendo,5.18,5.04,5.24,0.82,16.38
28,"Halo 3",X360,2007,Shooter,"Microsoft Game Studios",8.84,4.44,0.39,1.39,15.07
29,"Pokemon HeartGold/Pokemon SoulSilver",DS,2009,"Role-Playing",Nintendo,5.58,3.6,4.48,0.84,14.51
30,"Pokemon Black/Pokemon White",DS,2010,"Role-Playing",Nintendo,5.57,3.28,5.65,0.82,15.32
31,"Gran Turismo 4",PS2,2004,Racing,"Sony Computer Entertainment",3.46,5.14,1.12,0.36,10.09
32,"Super Mario 64",N64,1996,Platform,Nintendo,6.91,2.85,1.91,0.23,11.91
33,"Grand Theft Auto: Vice City",PS2,2002,Action,"Take-Two Interactive",8.41,5.17,0.31,0.89,14.79
34,"Pokemon Ruby/Pokemon Sapphire",GBA,2002,"Role-Playing",Nintendo,6.06,3.9,5.38,0.64,15.99
35,"New Super Mario Bros. 2",3DS,2012,Platform,Nintendo,4.49,3.31,4.7,1.09,13.59
36,"Grand Theft Auto III",PS2,2001,Action,"Take-Two Interactive",8.49,5.18,0.29,0.81,14.78
37,"Call of Duty: Black Ops",PS3,2010,Shooter,Activision,5.79,5.09,0.36,1.17,12.41
38,"Super Smash Bros. Brawl",Wii,2008,Fighting,Nintendo,6.62,2.5,2.33,0.66,12.11
39,"Call of Duty: Black Ops 2",PS3,2012,Shooter,Activision,4.56,4.08,0.25,1.12,10.01
40,"Pokemon X/Pokemon Y",3DS,2013,"Role-Playing",Nintendo,5.36,4.22,4.46,1.38,15.42
41,"Call of Duty: Modern Warfare 2",X360,2009,Shooter,Activision,7.04,3.29,0.08,0.88,11.3
42,"Grand Theft Auto IV",PS3,2008,Action,"Take-Two Interactive",5.17,5.27,0.29,1.06,11.79
43,"Super Mario Galaxy",Wii,2007,Platform,Nintendo,5.79,3.36,1.38,0.78,11.32
44,"Just Dance 3",Wii,2011,Misc,Ubisoft,5.21,3.68,0.09,0.69,9.68
45,"Call of Duty: Black Ops",X360,2010,Shooter,Activision,8,2.94,0.04,0.82,11.8
46,"Gran Turismo 5",PS3,2010,Racing,"Sony Computer Entertainment",2.81,4.14,1.25,0.51,8.71
47,"Mario Kart 7",3DS,2011,Racing,Nintendo,4.53,2.86,2.12,0.84,10.35
48,"Call of Duty: Modern Warfare 3",PS3,2011,Shooter,Activision,4.94,3.54,0.13,0.84,9.46
49,"Halo: Reach",X360,2010,Shooter,"Microsoft Game Studios",6.48,2.3,0.09,0.61,9.49
50,"Just Dance 2",Wii,2010,Misc,Ubisoft,4.35,3.65,0.04,0.56,8.61
51,"Call of Duty: Modern Warfare 2",PS3,2009,Shooter,Activision,4.95,3.56,0.11,0.76,9.39
52,"Grand Theft Auto IV",X360,2008,Action,"Take-Two Interactive",6.74,3.16,0.06,0.71,10.68
53,"Halo 2",Xbox,2004,Shooter,"Microsoft Game Studios",6.82,2.5,0.06,0.52,9.91
54,"FIFA 16",PS4,2015,Sports,"Electronic Arts",1.54,5.92,0.12,1.05,8.63
55,"Mario Kart 64",N64,1996,Racing,Nintendo,5.55,1.78,2.27,0.3,9.9
56,"Super Mario Land 2: 6 Golden Coins",GB,1992,Platform,Nintendo,6.37,1.64,2.42,0.27,10.7
57,"FIFA 17",PS4,2016,Sports,"Electronic Arts",1.26,5.72,0.11,0.99,8.08
58,"Gran Turismo",PS,1997,Racing,"Sony Computer Entertainment",3.53,4.07,2.88,0.27,10.75
59,"Super Mario Bros. 2",NES,1988,Platform,Nintendo,5.24,0.91,2.25,0.24,8.65
60,"GoldenEye 007",N64,1997,Shooter,Nintendo,6.09,2.29,0.31,0.43,9.12
61,"FIFA 15",PS4,2014,Sports,"Electronic Arts",1.62,4.98,0.08,0.87,7.56
62,"FIFA 14",PS3,2013,Sports,"Electronic Arts",1.3,4.78,0.08,0.84,7
63,"FIFA 13",PS3,2012,Sports,"Electronic Arts",1.41,4.57,0.06,0.76,6.8
64,"Call of Duty: Ghosts",X360,2013,Shooter,Activision,5.17,1.77,0.03,0.58,7.55
65,"Pokemon Yellow: Special Pikachu Edition",GB,1998,"Role-Playing",Nintendo,5.89,2.46,3.17,0.38,11.9
66,"The Elder Scrolls V: Skyrim",X360,2011,"Role-Playing","Bethesda Softworks",4.25,2.73,0.11,0.64,7.74
67,"Call of Duty: Black Ops 3",PS4,2015,Shooter,Activision,2.68,2.93,0.15,0.79,6.55
68,"Grand Theft Auto V",PS4,2014,Action,"Take-Two Interactive",3.03,2.92,0.24,0.77,6.96
69,"Super Mario 3D Land",3DS,2011,Platform,Nintendo,4.12,2.14,2.62,0.61,9.5
70,"Call of Duty: Black Ops 3",XOne,2015,Shooter,Activision,3.59,1.91,0.04,0.54,6.08
71,"Minecraft: PlayStation 3 Edition",PS3,2013,Misc,"Sony Computer Entertainment",2.58,2.49,0.09,0.53,5.69
72,"Call of Duty: Advanced Warfare",PS4,2014,Shooter,Activision,2.63,2.37,0.13,0.67,5.8
73,"Super Mario All-Stars",SNES,1993,Platform,Nintendo,5.15,1.9,2.38,0.28,9.71
74,"The Sims 3",PC,2009,Simulation,"Electronic Arts",1.94,3.16,0.01,0.71,5.82
75,"Need for Speed: Most Wanted",PS2,2005,Racing,"Electronic Arts",2.6,2.55,0.11,0.49,5.75
76,"Call of Duty: Modern Warfare 2",PS3,2009,Shooter,Activision,3.84,2.07,0.04,0.55,6.5
77,"New Super Mario Bros. U",WiiU,2012,Platform,Nintendo,3.73,1.98,0.96,0.54,7.21
78,"Super Mario Sunshine",GC,2002,Platform,Nintendo,4.02,1.78,0.89,0.39,7.08
79,"Mario Party 8",Wii,2007,Misc,Nintendo,4.48,1.66,0.46,0.42,7.03
80,"Pokemon Omega Ruby/Pokemon Alpha Sapphire",3DS,2014,"Role-Playing",Nintendo,3.31,1.76,2.5,0.61,8.18
81,"Call of Duty: World at War",X360,2008,Shooter,Activision,4.31,1.62,0.03,0.46,6.43
82,"The Legend of Zelda: Ocarina of Time",N64,1998,Adventure,Nintendo,4.12,1.69,1.14,0.31,7.26
83,"Donkey Kong Country",SNES,1994,Platform,Nintendo,4.03,1.71,1.12,0.3,7.16
84,"Call of Duty: Black Ops 2",X360,2012,Shooter,Activision,4.25,1.58,0.02,0.45,6.31
85,"Battlefield 3",PS3,2011,Shooter,"Electronic Arts",2.22,2.37,0.06,0.54,5.19
86,"Pokemon FireRed/Pokemon LeafGreen",GBA,2004,"Role-Playing",Nintendo,3.49,1.65,2.33,0.39,7.86
87,"Super Mario Galaxy 2",Wii,2010,Platform,Nintendo,3.61,1.56,0.79,0.4,6.36
88,"Super Mario 3D World",WiiU,2013,Platform,Nintendo,3.27,1.52,0.85,0.39,6.03
89,"Call of Duty 4: Modern Warfare",X360,2007,Shooter,Activision,4.02,1.4,0.03,0.39,5.85
90,"Grand Theft Auto: Liberty City Stories",PSP,2005,Action,"Take-Two Interactive",2.53,1.89,0.06,0.41,4.89
91,"The Legend of Zelda: Twilight Princess",Wii,2006,Adventure,Nintendo,3.46,1.46,0.57,0.33,5.82
92,"Animal Crossing: New Leaf",3DS,2012,Simulation,Nintendo,2.67,1.39,2.65,0.49,7.2
93,"Pokemon Platinum",DS,2008,"Role-Playing",Nintendo,2.99,1.61,2.43,0.46,7.49
94,"Need for Speed: Underground",PS2,2003,Racing,"Electronic Arts",2.34,1.79,0.06,0.38,4.57
95,"Grand Theft Auto: Chinatown Wars",DS,2009,Action,"Take-Two Interactive",2.27,1.58,0.07,0.35,4.27
96,"Super Mario 64 DS",DS,2004,Platform,Nintendo,2.85,1.5,1.63,0.37,6.35
97,"Call of Duty: Black Ops 4",PS4,2018,Shooter,Activision,2.08,1.71,0.13,0.58,4.5
98,"Pokemon Emerald",GBA,2004,"Role-Playing",Nintendo,2.8,1.47,2.03,0.34,6.64
99,"Pokemon Black 2/Pokemon White 2",DS,2012,"Role-Playing",Nintendo,2.47,1.36,2.25,0.39,6.47
100,"Call of Duty: WWII",PS4,2017,Shooter,Activision,2.01,1.6,0.12,0.56,4.29
101,"The Legend of Zelda: Breath of the Wild",Switch,2017,Adventure,Nintendo,3.84,2.07,1.29,0.61,7.81
102,"Pokemon Sun/Pokemon Moon",3DS,2016,"Role-Playing",Nintendo,2.93,1.68,3.04,0.58,8.23
103,"Super Smash Bros. for Wii U",WiiU,2014,Fighting,Nintendo,2.72,1.39,1.08,0.38,5.57
104,"Call of Duty: Infinite Warfare",PS4,2016,Shooter,Activision,1.97,1.53,0.11,0.53,4.14
105,"Red Dead Redemption 2",PS4,2018,Action,"Take-Two Interactive",2.13,1.68,0.19,0.56,4.56
106,"Super Mario Odyssey",Switch,2017,Platform,Nintendo,3.59,1.89,1.71,0.58,7.77
107,"Spider-Man",PS4,2018,Action,"Sony Interactive Entertainment",2.74,1.77,0.14,0.59,5.24
108,"God of War",PS4,2018,Action,"Sony Interactive Entertainment",2.45,1.67,0.11,0.53,4.76
109,"Mario Kart 8 Deluxe",Switch,2017,Racing,Nintendo,3.24,1.67,1.36,0.52,6.79
110,"Grand Theft Auto V",PC,2015,Action,"Take-Two Interactive",1.68,1.85,0.04,0.48,4.05
111,"The Legend of Zelda: Skyward Sword",Wii,2011,Adventure,Nintendo,2.88,1.29,0.58,0.31,5.06
112,"Mario Party DS",DS,2007,Misc,Nintendo,2.53,1.21,0.93,0.28,4.95
113,"Super Mario Maker",3DS,2016,Platform,Nintendo,2.07,1.15,1.45,0.33,5
114,"Pokemon Ultra Sun/Pokemon Ultra Moon",3DS,2017,"Role-Playing",Nintendo,2.12,1.12,2.05,0.37,5.66
115,"Uncharted 4: A Thief's End",PS4,2016,Action,"Sony Interactive Entertainment",1.97,1.46,0.1,0.48,4.01
116,"The Witcher 3: Wild Hunt",PS4,2015,"Role-Playing","Warner Bros. Interactive Entertainment",1.89,1.52,0.1,0.45,3.96
117,"Minecraft: Xbox 360 Edition",X360,2012,Misc,"Microsoft Studios",2.45,1.21,0.03,0.34,4.03
118,"Call of Duty: Modern Warfare Remastered",PS4,2016,Shooter,Activision,1.94,1.34,0.09,0.44,3.81
119,"FIFA 18",PS4,2017,Sports,"Electronic Arts",1.44,1.7,0.08,0.46,3.68
120,"FIFA 19",PS4,2018,Sports,"Electronic Arts",1.28,1.65,0.07,0.43,3.43
121,"The Last of Us",PS4,2014,Action,"Sony Computer Entertainment",1.76,1.33,0.09,0.42,3.6
122,"Super Mario Party",Switch,2018,Misc,Nintendo,2.23,1.13,0.87,0.32,4.55
123,"Call of Duty: Black Ops 4",XOne,2018,Shooter,Activision,1.92,1.18,0.03,0.35,3.48
124,"Super Mario Run",iOS,2016,Platform,Nintendo,1.85,1.14,0.59,0.28,3.86
125,"Luigi's Mansion 3",Switch,2019,Adventure,Nintendo,2.11,1.08,0.81,0.31,4.31
126,"Animal Crossing: New Horizons",Switch,2020,Simulation,Nintendo,5.32,3.52,2.46,1.05,12.35
127,"Call of Duty: Modern Warfare",PS4,2019,Shooter,Activision,2.78,1.68,0.14,0.57,5.17
128,"The Legend of Zelda: Link's Awakening",Switch,2019,Adventure,Nintendo,1.95,1.12,0.72,0.28,4.07
129,"Pokemon Sword/Pokemon Shield",Switch,2019,"Role-Playing",Nintendo,3.12,1.89,1.85,0.61,7.47
130,"Super Smash Bros. Ultimate",Switch,2018,Fighting,Nintendo,3.84,1.98,1.91,0.63,8.36
131,"Mario Kart Live: Home Circuit",Switch,2020,Racing,Nintendo,1.23,0.78,0.42,0.19,2.62
132,"Pokemon: Let's Go, Pikachu!/Let's Go, Eevee!",Switch,2018,"Role-Playing",Nintendo,2.67,1.41,1.34,0.45,5.87
133,"Super Mario 3D All-Stars",Switch,2020,Platform,Nintendo,2.45,1.28,1.12,0.39,5.24
134,"Paper Mario: The Origami King",Switch,2020,"Role-Playing",Nintendo,1.56,0.83,0.61,0.22,3.22
135,"Pikmin 3 Deluxe",Switch,2020,Strategy,Nintendo,1.34,0.71,0.48,0.17,2.7
136,"Xenoblade Chronicles: Definitive Edition",Switch,2020,"Role-Playing",Nintendo,1.12,0.59,0.45,0.15,2.31
137,"Hyrule Warriors: Age of Calamity",Switch,2020,Action,Nintendo,1.45,0.77,0.58,0.21,3.01
138,"New Pokemon Snap",Switch,2021,Simulation,Nintendo,1.23,0.65,0.52,0.18,2.58
139,"Super Mario 3D World + Bowser's Fury",Switch,2021,Platform,Nintendo,2.34,1.23,0.94,0.34,4.85
140,"The Legend of Zelda: Skyward Sword HD",Switch,2021,Adventure,Nintendo,1.56,0.82,0.63,0.23,3.24
141,"WarioWare: Get It Together!",Switch,2021,Puzzle,Nintendo,1.01,0.53,0.45,0.16,2.15
142,"Metroid Dread",Switch,2021,Adventure,Nintendo,1.34,0.7,0.56,0.2,2.8
143,"Pokemon Brilliant Diamond/Shining Pearl",Switch,2021,"Role-Playing",Nintendo,2.89,1.52,1.45,0.52,6.38
144,"Advance Wars 1+2: Re-Boot Camp",Switch,2023,Strategy,Nintendo,0.89,0.47,0.38,0.14,1.88
145,"Mario Strikers: Battle League",Switch,2022,Sports,Nintendo,1.23,0.65,0.52,0.18,2.58
146,"Xenoblade Chronicles 3",Switch,2022,"Role-Playing",Nintendo,1.56,0.82,0.63,0.23,3.24
147,"Splatoon 3",Switch,2022,Shooter,Nintendo,2.34,1.23,1.89,0.45,5.91
148,"Bayonetta 3",Switch,2022,Action,Nintendo,1.12,0.59,0.45,0.15,2.31
149,"Kirby and the Forgotten Land",Switch,2022,Platform,Nintendo,1.89,0.99,0.78,0.28,3.94
150,"Fire Emblem Engage",Switch,2023,Strategy,Nintendo,1.34,0.7,0.56,0.2,2.8
151,"The Legend of Zelda: Tears of the Kingdom",Switch,2023,Adventure,Nintendo,4.56,2.41,1.89,0.68,9.54
152,"Super Mario Bros. Wonder",Switch,2023,Platform,Nintendo,3.21,1.69,1.34,0.48,6.72
153,Starfield,"Xbox Series X",2023,"Role-Playing","Bethesda Softworks",2.34,1.23,0.12,0.42,4.11
154,"Spider-Man 2",PS5,2023,Action,"Sony Interactive Entertainment",2.89,1.52,0.15,0.51,5.07
155,"Baldur's Gate 3",PC,2023,"Role-Playing","Larian Studios",1.56,1.23,0.08,0.34,3.21
156,"Resident Evil 4",PS5,2023,Action,Capcom,1.89,1.12,0.45,0.28,3.74
157,"Hogwarts Legacy",PS5,2023,Action,"Warner Bros. Interactive Entertainment",3.12,1.89,0.23,0.61,5.85
158,"Final Fantasy XVI",PS5,2023,"Role-Playing","Square Enix",1.34,0.89,0.78,0.2,3.21
159,"Street Fighter 6",PS5,2023,Fighting,Capcom,1.23,0.78,0.52,0.18,2.71
160,"Mortal Kombat 1",PS5,2023,Fighting,"Warner Bros. Interactive Entertainment",1.56,0.82,0.12,0.23,2.73
161,"Diablo IV",PS5,2023,"Role-Playing","Blizzard Entertainment",2.34,1.23,0.15,0.42,4.14
162,"Armored Core VI: Fires of Rubicon",PS5,2023,Action,"Bandai Namco Entertainment",1.12,0.65,0.45,0.15,2.37
163,"Like a Dragon: Infinite Wealth",PS5,2024,"Role-Playing",SEGA,0.89,0.47,0.38,0.14,1.88
164,"Persona 3 Reload",PS5,2024,"Role-Playing",Atlus,0.78,0.41,0.34,0.12,1.65
165,"Final Fantasy VII Rebirth",PS5,2024,"Role-Playing","Square Enix",1.23,0.65,0.52,0.18,2.58
166,"Prince of Persia: The Lost Crown",PS5,2024,Action,Ubisoft,0.67,0.35,0.28,0.1,1.4
167,"Tekken 8",PS5,2024,Fighting,"Bandai Namco Entertainment",0.89,0.47,0.38,0.14,1.88
168,"Helldivers 2",PS5,2024,Shooter,"Sony Interactive Entertainment",1.34,0.7,0.12,0.23,2.39
169,"Rise of the Ronin",PS5,2024,Action,"Sony Interactive Entertainment",0.78,0.41,0.34,0.12,1.65
170,"Dragon's Dogma 2",PS5,2024,"Role-Playing",Capcom,1.12,0.59,0.45,0.15,2.31
171,"Stellar Blade",PS5,2024,Action,"Sony Interactive Entertainment",0.89,0.47,0.38,0.14,1.88
172,"Elden Ring",PS5,2022,Action,"Bandai Namco Entertainment",3.45,2.12,0.89,0.61,7.07
173,"God of War Ragnarök",PS5,2022,Action,"Sony Interactive Entertainment",3.12,1.89,0.23,0.61,5.85
174,"Harry Potter: Quidditch Champions",PS5,2023,Sports,"Warner Bros. Interactive Entertainment",0.56,0.29,0.23,0.08,1.16
175,"Assassin's Creed Mirage",PS5,2023,Action,Ubisoft,1.23,0.65,0.12,0.23,2.23
176,"Call of Duty: Modern Warfare III",PS5,2023,Shooter,Activision,2.34,1.23,0.15,0.42,4.14
177,"Avatar: Frontiers of Pandora",PS5,2023,Action,Ubisoft,1.12,0.59,0.12,0.23,2.06
178,"Alan Wake II",PS5,2023,Action,"Epic Games",0.89,0.47,0.08,0.17,1.61
179,"Cyberpunk 2077: Phantom Liberty",PS5,2023,"Role-Playing","CD Projekt",1.56,0.82,0.15,0.28,2.81
180,"Marvel's Spider-Man: Miles Morales",PS5,2020,Action,"Sony Interactive Entertainment",2.34,1.23,0.23,0.42,4.22
181,"Ratchet & Clank: Rift Apart",PS5,2021,Platform,"Sony Interactive Entertainment",1.89,0.99,0.19,0.34,3.41
182,Returnal,PS5,2021,Shooter,"Sony Interactive Entertainment",1.23,0.65,0.12,0.23,2.23
183,Deathloop,PS5,2021,Action,"Bethesda Softworks",0.89,0.47,0.08,0.17,1.61
184,"Ghostwire: Tokyo",PS5,2022,Action,"Bethesda Softworks",0.78,0.41,0.34,0.12,1.65
185,Forspoken,PS5,2023,Action,"Square Enix",0.67,0.35,0.28,0.1,1.4
186,"Final Fantasy XIV: Endwalker",PS5,2021,"Role-Playing","Square Enix",1.12,0.59,0.45,0.15,2.31
187,"Gran Turismo 7",PS5,2022,Racing,"Sony Interactive Entertainment",1.89,0.99,0.78,0.28,3.94
188,"Hogwarts Legacy",Switch,2023,Action,"Warner Bros. Interactive Entertainment",1.56,0.82,0.63,0.23,3.24
189,"The Witcher 3: Wild Hunt - Complete Edition",Switch,2019,"Role-Playing","CD Projekt",1.23,0.65,0.52,0.18,2.58
190,"BioShock: The Collection",Switch,2020,Shooter,"2K Games",0.89,0.47,0.38,0.14,1.88
191,"Borderlands Legendary Collection",Switch,2020,Shooter,"2K Games",0.78,0.41,0.34,0.12,1.65
192,"Crash Bandicoot N. Sane Trilogy",Switch,2018,Platform,Activision,1.56,0.82,0.63,0.23,3.24
193,Cuphead,Switch,2019,Shooter,"Studio MDHR",1.23,0.65,0.52,0.18,2.58
194,"Dead Cells",Switch,2018,Action,"Motion Twin",0.89,0.47,0.38,0.14,1.88
195,"Diablo III: Eternal Collection",Switch,2018,"Role-Playing","Blizzard Entertainment",1.56,0.82,0.63,0.23,3.24
196,"Doom Eternal",Switch,2020,Shooter,"Bethesda Softworks",1.23,0.65,0.52,0.18,2.58
197,"Dragon Quest XI S: Echoes of an Elusive Age",Switch,2019,"Role-Playing","Square Enix",0.89,0.47,0.38,0.14,1.88
198,Hades,Switch,2020,Action,"Supergiant Games",1.56,0.82,0.63,0.23,3.24
199,"Hollow Knight",Switch,2018,Adventure,"Team Cherry",1.23,0.65,0.52,0.18,2.58
200,"Into the Breach",Switch,2018,Strategy,Subset,0.89,0.47,0.38,0.14,1.88"""

import pandas as pd
import io

pdf = pd.read_csv(io.StringIO(csv_200))
df = spark.createDataFrame(pdf)

print(f"DataFrame created with {df.count():,} records")
print(f"Columns: {df.columns}")
print(f"Dataset spans from {df.select('Year').agg({'Year': 'min'}).collect()[0][0]} to {df.select('Year').agg({'Year': 'max'}).collect()[0][0]}")

print("\n First 5 Rows:")
df.show(5, truncate=False)

print("\nLast 5 Rows:")
df.orderBy(col("Rank").desc()).show(5, truncate=False)

print("\nBasic Statistics:")
print(f"• Unique Platforms: {df.select('Platform').distinct().count()}")
print(f"• Unique Genres: {df.select('Genre').distinct().count()}")
print(f"• Unique Publishers: {df.select('Publisher').distinct().count()}")
print(f"• Years covered: {df.select('Year').distinct().count()}")
print(f"• Total Global Sales: {df.select(sum('Global_Sales')).collect()[0][0]:,.2f} million units")

In [0]:
print("DATA CLEANING AND PREPARATION")

pdf = pd.read_csv(io.StringIO(csv_200))

numeric_cols = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales"]
for col in numeric_cols:
    pdf[col] = pd.to_numeric(pdf[col], errors='coerce')
pdf["Year"] = pd.to_numeric(pdf["Year"], errors='coerce')

df_clean = spark.createDataFrame(pdf)

print(f"Data cleaning completed!")
print(f"Clean records: {df_clean.count():,}")

df_clean.write.mode("overwrite").saveAsTable("video_games_clean")
print("Clean data saved as 'video_games_clean' Delta table")

print("\nCleaned Data Sample:")
df_clean.select("Rank", "Name", "Platform", "Year", "Genre", "Global_Sales") \
        .orderBy("Rank") \
        .limit(10) \
        .show(truncate=False)

print("\nCleaned Data Schema:")
df_clean.printSchema()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, avg, round, sum, min, max, desc

spark = SparkSession.builder.getOrCreate()

print("\n" + "=" * 70)
print("CREATING ANALYTICAL TABLES")
print("=" * 70)

publisher_stats = df_clean.groupBy("Publisher") \
    .agg(
        count("*").alias("games_count"),
        round(sum("Global_Sales"), 2).alias("total_sales_millions"),
        round(avg("Global_Sales"), 2).alias("avg_sales_per_game"),
        round(sum("NA_Sales"), 2).alias("na_sales"),
        round(sum("EU_Sales"), 2).alias("eu_sales"),
        round(sum("JP_Sales"), 2).alias("jp_sales")
    ) \
    .filter(col("total_sales_millions") > 10) \
    .orderBy(desc("total_sales_millions"))

publisher_stats.write.mode("overwrite").saveAsTable("publisher_performance")
print("Publisher performance table created")

genre_trends = df_clean.groupBy("Year", "Genre") \
    .agg(
        count("*").alias("games_released"),
        round(sum("Global_Sales"), 2).alias("total_sales"),
        round(avg("Global_Sales"), 2).alias("avg_sales")
    ) \
    .filter(col("Year").isNotNull()) \
    .orderBy("Year", desc("total_sales"))

genre_trends.write.mode("overwrite").saveAsTable("genre_trends")
print("Genre trends table created")

platform_stats = df_clean.groupBy("Platform") \
    .agg(
        count("*").alias("games_count"),
        round(sum("Global_Sales"), 2).alias("total_sales"),
        min("Year").alias("first_year"),
        max("Year").alias("last_year"),
        round(avg("Global_Sales"), 2).alias("avg_sales")
    ) \
    .orderBy(desc("total_sales"))

platform_stats.write.mode("overwrite").saveAsTable("platform_analysis")
print("Platform analysis table created")

print("\nTop Publishers by Sales:")
publisher_stats.select("Publisher", "games_count", "total_sales_millions") \
               .show(10, truncate=False)

In [0]:
print("\n" + "=" * 70)
print("SQL ANALYTICAL QUERIES")
print("=" * 70)

df_clean.createOrReplaceTempView("video_games")

query1 = """
SELECT 
    Name,
    Platform,
    Year,
    Genre,
    Publisher,
    Global_Sales,
    NA_Sales,
    EU_Sales,
    JP_Sales,
    Other_Sales,
    ROW_NUMBER() OVER (ORDER BY Global_Sales DESC) as global_rank,
    ROW_NUMBER() OVER (PARTITION BY Platform ORDER BY Global_Sales DESC) as platform_rank,
    CASE 
        WHEN NA_Sales > EU_Sales AND NA_Sales > JP_Sales THEN 'North America'
        WHEN EU_Sales > NA_Sales AND EU_Sales > JP_Sales THEN 'Europe'
        WHEN JP_Sales > NA_Sales AND JP_Sales > EU_Sales THEN 'Japan'
        ELSE 'Other Regions'
    END as top_region
FROM video_games
WHERE Year >= 2000
ORDER BY Global_Sales DESC
LIMIT 20
"""

print("\nQuery 1: Top Performing Games Analysis")
top_games = spark.sql(query1)
top_games.show(truncate=False)

query2 = """
WITH yearly_genre_sales AS (
    SELECT 
        Year,
        Genre,
        SUM(Global_Sales) as total_sales,
        LAG(SUM(Global_Sales), 1) OVER (PARTITION BY Genre ORDER BY Year) as prev_year_sales
    FROM video_games
    WHERE Year IS NOT NULL 
      AND Year BETWEEN 2000 AND 2016
    GROUP BY Year, Genre
    HAVING SUM(Global_Sales) > 5
)
SELECT 
    Year,
    Genre,
    ROUND(total_sales, 2) as current_year_sales,
    ROUND(prev_year_sales, 2) as previous_year_sales,
    ROUND(
        CASE 
            WHEN prev_year_sales > 0 
            THEN ((total_sales - prev_year_sales) / prev_year_sales) * 100
            ELSE NULL 
        END, 2
    ) as growth_percentage,
    RANK() OVER (PARTITION BY Year ORDER BY total_sales DESC) as genre_rank_year
FROM yearly_genre_sales
WHERE Year >= 2005
ORDER BY Year DESC, total_sales DESC
LIMIT 30
"""

print("\nQuery 2: Year-over-Year Genre Sales Growth")
genre_growth = spark.sql(query2)
genre_growth.show(30, truncate=False)

top_games.write.mode("overwrite").saveAsTable("top_performing_games")
genre_growth.write.mode("overwrite").saveAsTable("genre_sales_growth")
print("\nSQL query results saved as tables for dashboard")

In [0]:
print(" PYSPARK DATA ANALYSIS (NO DESC FUNCTION)")

games_df = spark.table("video_games_clean")

games_df.select("Global_Sales", "NA_Sales", "EU_Sales", "JP_Sales").summary().show()
top_games = games_df.select("Name", "Platform", "Year", "Genre", "Publisher", "Global_Sales") \
                    .sort(col("Global_Sales").desc()) \
                    .limit(10)
top_games.show(truncate=False)

print("\n3. Publisher Market Share Analysis:")
publisher_share = games_df.groupBy("Publisher") \
    .agg(
        round(sum("Global_Sales"), 2).alias("total_sales"),
        count("*").alias("games_count")
    ) \
    .filter(col("total_sales") > 10) \
    .sort(col("total_sales").desc())

total_sales = games_df.select(sum("Global_Sales")).collect()[0][0]
publisher_share = publisher_share.withColumn(
    "market_share_pct",
    round((col("total_sales") / total_sales) * 100, 2)
)

publisher_share.show(10, truncate=False)

print("\n4. Sales Trends Over Years:")
yearly_sales = games_df.filter(col("Year").isNotNull()) \
    .groupBy("Year") \
    .agg(
        count("*").alias("games_released"),
        round(sum("Global_Sales"), 2).alias("total_sales"),
        round(avg("Global_Sales"), 2).alias("avg_sales_per_game")
    ) \
    .filter(col("Year") >= 1980) \
    .sort("Year")

from pyspark.sql.window import Window
window_spec = Window.orderBy(col("total_sales").desc())
peak_row = yearly_sales.withColumn("rank", row_number().over(window_spec)).filter(col("rank") == 1).first()
if peak_row:
    print(f" Peak Sales Year: {peak_row['Year']} with ${peak_row['total_sales']}M in sales")

yearly_sales.show(10, truncate=False)

print("\n5. Genre Performance Analysis:")
genre_performance = games_df.groupBy("Genre") \
    .agg(
        count("*").alias("games_count"),
        round(sum("Global_Sales"), 2).alias("total_sales"),
        round(avg("Global_Sales"), 2).alias("avg_sales"),
        round(min("Global_Sales"), 2).alias("min_sales"),
        round(max("Global_Sales"), 2).alias("max_sales")
    ) \
    .sort(col("total_sales").desc())

genre_performance.show(truncate=False)

print("\n 6. Regional Preferences Analysis:")
region_preferences = games_df.groupBy("Genre") \
    .agg(
        round(sum("NA_Sales"), 2).alias("na_total"),
        round(sum("EU_Sales"), 2).alias("eu_total"),
        round(sum("JP_Sales"), 2).alias("jp_total"),
        round(sum("Other_Sales"), 2).alias("other_total")
    ) \
    .withColumn("top_region", 
                when((col("na_total") > col("eu_total")) & (col("na_total") > col("jp_total")), "North America")
                .when((col("eu_total") > col("na_total")) & (col("eu_total") > col("jp_total")), "Europe")
                .when((col("jp_total") > col("na_total")) & (col("jp_total") > col("eu_total")), "Japan")
                .otherwise("Other")) \
    .sort(col("na_total").desc())

region_preferences.show(truncate=False)

genre_performance.write.mode("overwrite").saveAsTable("genre_performance_analysis")
yearly_sales.write.mode("overwrite").saveAsTable("yearly_sales_trends")
print("\nAnalysis results saved to Delta tables")

In [0]:
print("DATABRICKS SQL DASHBOARD CONFIGURATION")

print("\nAll tables are ready for dashboard creation!")
print("Tables available:")
spark.sql("SHOW TABLES").show(truncate=False)